In [30]:
import sqlite3
import MeCab

# MeCabオブジェクトを初期化
mecab = MeCab.Tagger("-Owakati")

# データベースに接続
conn = sqlite3.connect(":memory:")
c = conn.cursor()

# FTS5テーブルを作成 (contentカラムに加えて、filepathとnumberカラムを追加)
c.execute("CREATE VIRTUAL TABLE docs USING fts5(content, filepath, number);")
#c.execute("CREATE VIRTUAL TABLE docs USING fts5(content UNINDEXED, filepath, number);")
# ファイルパスと番号を含む文書を追加
documents = [
    ("日本語のテキスト", "/path/to/file1", 1),
    ("検索エンジンの仕組み", "/path/to/file2", 2),
    ("pythonとは何か", "/path/to/file3", 3),
    ("藤田です", "/path/to/file4", 4),
]
for doc, filepath, number in documents:
    doc_wakati = mecab.parse(doc).strip()
    c.execute("INSERT INTO docs(content, filepath, number) VALUES(?, ?, ?);", (doc_wakati, filepath, number))

# コミット
conn.commit()


In [49]:
# クエリ（ここでは "テキスト検索できますか"）
query = "テキスト検索できますか"
#query="藤田先生の業績は"
query_wakati = mecab.parse(query).strip()

# 単語ごとにクエリを作成
query_words = query_wakati.split()

# 各単語で検索を行い、結果を組み合わせる
combined_results = []
for query_word in query_words:
    #c.execute("SELECT content, bm25(docs) FROM docs WHERE docs MATCH ? ORDER BY bm25(docs);", (query_word,))
    c.execute("SELECT content, filepath, bm25(docs) FROM docs WHERE docs MATCH ? ORDER BY bm25(docs);", (query_word,))
    results = c.fetchall()
    combined_results.extend(results)

# 各文書のスコアを合計
score_dict = {}
for result in combined_results:
    if result[0] in score_dict:
        score_dict[result[0]] += result[2]
    else:
        score_dict[result[0]] = result[2]

# 結果をスコアでソート
sorted_results = sorted(score_dict.items(), key=lambda x: x[1])

ret_list=[]
for result in sorted_results:
    key=result[0]
    for record in combined_results:
        if record[0]==key:
            break
    temp_dict = {}
    temp_dict["path"] = record[1]
    temp_dict["text"] = record[0]
    temp_dict["sim"] = record[2]
    ret_list.append(temp_dict)

for ret in ret_list:
    print(ret["text"],ret["path"], ret["sim"])

日本 語 の テキスト /path/to/file1 -0.8362621429581374
検索 エンジン の 仕組み /path/to/file2 -0.8362621429581374
python と は 何 か /path/to/file3 -0.7948516379423286
